In [ ]:
ROI_NAMES = ['patient','lung','gtv'] #Will be used to match the ROINames in the RTSTRUCT file
ROOT_PATH = r"C:\Users\sithi\Research\dataset\Lung1_organized"
OUT_PATH = r"C:\Users\sithi\Research\dataset_processed\Lung1_npy_vol"

In [ ]:
import os as os
import pydicom as pydicom
import SimpleITK as sitk
import numpy as np
from skimage.draw import polygon
from concurrent.futures import ThreadPoolExecutor

In [ ]:
from tqdm import tqdm
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt

# DICOM Series & RTSTRUCT to npy volumes

In [ ]:
def dcms_to_npy(path, pbar=tqdm([1], position=0,desc="Processing dcm_files")):
    
    pbar.set_description(f"Loading dcm series inside folder {path}")
    
    try:
        series_ids = sitk.ImageSeriesReader.GetGDCMSeriesIDs(path)
    except:
        pbar.set_description(f"No dcm series found in path {path}")
        return
    
    img_vol = []
    seen_files = set()
    out_path = path.replace(ROOT_PATH, OUT_PATH)
    img_position, pixel_spacing, slope, intercept = None, None, None, None
    candidate_rt_paths = set()



    for id in series_ids:

        dcm_files = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(path, id)
        seen_files = seen_files.union([os.path.join(path,file.split('/')[-1]) for file in dcm_files])

        for file in dcm_files:
            dcm_file = pydicom.dcmread(file)
            if not (dcm_file.Modality == 'CT'):
                break;

            if not (img_vol):
                img_position = np.array(dcm_file.ImagePositionPatient)
                pixel_spacing = np.array([*dcm_file.PixelSpacing] + [dcm_file.SliceThickness])
                slope = dcm_file.RescaleSlope
                intercept = dcm_file.RescaleIntercept

                keys = ["Manufacturer", "SoftwareVersions", "Kernel", "KVP", "ExposureTime", "Exposure",
                        "TubeCurrent", "PixelSpacing", "SliceThickness", "StudyInstanceUID", "PatientName",
                        "PatientID"]
                values = [dcm_file.get("Manufacturer",""), dcm_file.get("SoftwareVersions",""), dcm_file.get("ConvolutionKernel",""),
                          dcm_file.get("KVP",""),dcm_file.get("ExposureTime",""), dcm_file.get("Exposure",""), dcm_file.get("XRayTubeCurrent",""),
                          dcm_file.get("PixelSpacing",""), dcm_file.get("SliceThickness",""), dcm_file.get("StudyInstanceUID",""),
                          dcm_file.get("PatientName",""), dcm_file.get("PatientID","")]
                meta_info = dict(zip(keys, values))


                if not os.path.exists(out_path):
                    os.makedirs(out_path)

                np.save(os.path.join(out_path,"meta_info.npy"),meta_info)

            img_vol.append(dcm_file.pixel_array)

    all_files = set(os.path.join(path, file) for file in os.listdir(path) if file.endswith(".dcm"))
    candidate_rt_paths = candidate_rt_paths.union(all_files - seen_files)
    

    img_vol = np.array(img_vol)
    img_vol = (img_vol * slope) + intercept
    np.save(os.path.join(out_path, "image.npy"), img_vol)
        
        
    rt_flag = 0
        
    for rt_path in candidate_rt_paths:
        
        
        try:
            dcm = pydicom.dcmread(rt_path)
        except:
            rt_flag += 1
            continue;
            
        if not(dcm.Modality=='RTSTRUCT'):
            rt_flag += 1
            continue;
            

        out_path = os.path.join(out_path, "masks")
        if not os.path.exists(out_path):
            os.makedirs(out_path)
            

        for ROI_Name in ROI_NAMES:

            mask_vol = np.zeros_like(img_vol, dtype=np.bool8)
            
            connected_vol = np.zeros_like(img_vol, dtype=np.bool8)


            ROI_Numbers = [roi_sequence.ROINumber for roi_sequence in dcm.StructureSetROISequence]
            candidate_roi_indeces = [ROI_Numbers.index(roi_sequence.ROINumber) for roi_sequence in dcm.StructureSetROISequence if ROI_Name.lower() in roi_sequence.ROIName.lower()]
            
            if(candidate_roi_indeces):

                for index in candidate_roi_indeces:


                    coords_list = [sequence.ContourData for sequence in
                                   dcm.ROIContourSequence[index].ContourSequence]

                    _mask_vol = np.copy(mask_vol)

                    cache_indeces = []

                    for coords in coords_list:
                        coords = np.array(coords).reshape(-1, 3)

                        coords = np.rint((coords - img_position) / pixel_spacing)

                        Y = coords[:, 0]
                        X = coords[:, 1]
                        slice_no = int(np.unique(coords[:, 2])[0])
                        
                        if slice_no>=len(mask_vol):
                            
                            print(f"Ignored error while loading {ROI_Name} contour at Z-index {slice_no} in {path}")
                            continue;

                        rr, cc = polygon(X, Y)
                        _mask_vol[slice_no, rr, cc] = 1
                        
                        cache_indeces += [slice_no]


                    _connected_vol = np.copy(_mask_vol)
                    
                    #trivial connected component extraction, helps in minor contour correction

                    for slice_no in cache_indeces:


                        if slice_no == 0:
                            _connected_vol[slice_no] = _connected_vol[slice_no] | _connected_vol[slice_no + 1]
                        elif slice_no == len(_mask_vol) - 1:
                            _connected_vol[slice_no] = _connected_vol[slice_no - 1] | _connected_vol[slice_no]
                        else:
                            _connected_vol[slice_no] = (_connected_vol[slice_no] | _connected_vol[
                                slice_no + 1]) & (_connected_vol[slice_no - 1] | _connected_vol[slice_no])

                    mask_vol = mask_vol | _mask_vol
                    connected_vol = connected_vol | _connected_vol
                            
                
                #np.save(os.path.join(out_path, ROI_Name.lower()+".npy"), mask_vol)
                np.save(os.path.join(out_path,ROI_Name.lower()+".npy"),connected_vol)
     
    if rt_flag==len(candidate_rt_paths):
        print(f"No RTSTRUCT file found inside folder {path}")
        
    pbar.update()


__Sanity Check__

In [ ]:
test_folder_name = "Lung-LUNG1-307"

In [ ]:
path = os.path.join(ROOT_PATH,test_folder_name)
dcms_to_npy(path)

__Visualizing Volume__

In [ ]:
path = os.path.join(OUT_PATH,test_folder_name)
img_path = os.path.join(path,"image.npy")
mask_path = os.path.join(path,"masks")

img = np.load(img_path)
masks = np.array([np.load(os.path.join(mask_path,f"{roi_name.lower()}.npy")) for roi_name in ROI_NAMES])

def visualize_vol(i):
    
    plt.imshow(img[i],cmap='gray')
    _masks = masks[:,i]
    
    for mask,roi_name in zip(_masks, ROI_NAMES):
        plt.scatter(np.argwhere(mask)[:,1],np.argwhere(mask)[:,0], label=roi_name)
        #plt.contour(mask)
       
    plt.legend()
         
         
    

interact(visualize_vol,i=widgets.IntSlider(0,0,len(img)-1,1))

# Main

In [ ]:
num_workers = 3

In [ ]:
paths = [os.path.join(ROOT_PATH,path) for path in os.listdir(ROOT_PATH)]

with tqdm(paths, position=0, desc=f"Processing dcm files") as pbar:
    with ThreadPoolExecutor(num_workers) as e:e.map(dcms_to_npy,paths,[pbar]*len(paths))